<a href="https://colab.research.google.com/github/steliosg23/Data_Challenge_2025/blob/main/4_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the base directory (adjust this path to your directory)
base_dir = '/content/drive/MyDrive/Data Science AUEB/Data Challenge/data/'

train_df = pd.read_csv(f'{base_dir}final_df.csv')
test_df = pd.read_csv(f'{base_dir}test_df.csv')




Mounted at /content/drive


In [ ]:
# Inspect the first few rows to see the available columns
train_df

,product_id,text,price,node_degree,core_number,avg_neighbor_degree,class_label
0,0,fsa orbit z zero stack internal bicycle headse...,50.50,0.000036,8,70.700000,5
1,1,columbia bugaboo ii footbyfoot pole person dom...,54.30,0.000036,10,111.200000,10
2,2,men new gym workout short gary majdell sport,17.97,0.000036,10,202.200000,0
3,4,real avid zipwire pistol cleaning kit real avi...,36.60,0.000047,10,80.153846,2
4,5,harvard sport trainer harvard sport trainer le...,199.95,0.000040,10,44.636364,8
...,...,...,...,...,...,...,...
156399,276448,abu garcia c ambassadeur c baitcast round reel...,112.99,0.000043,10,115.166667,2
156400,276450,australian outrider airflow trayvil pad black ...,36.00,0.000022,4,7.666667,15
156401,276450,australian outrider airflow trail pad black ai...,36.00,0.000022,4,7.666667,15
156402,276450,australian outrider airflow pad black air flow...,36.00,0.000022,4,7.666667,15


In [ ]:
# prompt: drop rows where text is nan

train_df = train_df.dropna(subset=['text'])

train_df

,product_id,text,price,node_degree,core_number,avg_neighbor_degree,class_label
0,0,fsa orbit z zero stack internal bicycle headse...,50.50,0.000036,8,70.700000,5
1,1,columbia bugaboo ii footbyfoot pole person dom...,54.30,0.000036,10,111.200000,10
2,2,men new gym workout short gary majdell sport,17.97,0.000036,10,202.200000,0
3,4,real avid zipwire pistol cleaning kit real avi...,36.60,0.000047,10,80.153846,2
4,5,harvard sport trainer harvard sport trainer le...,199.95,0.000040,10,44.636364,8
...,...,...,...,...,...,...,...
156399,276448,abu garcia c ambassadeur c baitcast round reel...,112.99,0.000043,10,115.166667,2
156400,276450,australian outrider airflow trayvil pad black ...,36.00,0.000022,4,7.666667,15
156401,276450,australian outrider airflow trail pad black ai...,36.00,0.000022,4,7.666667,15
156402,276450,australian outrider airflow pad black air flow...,36.00,0.000022,4,7.666667,15


In [ ]:
test_df



,product_id,text,price,node_degree,core_number,avg_neighbor_degree
0,49957,Geigerrig Guardian Rig Tactical Hydration Pack...,133.00,0.000025,7,62.285714
1,135386,Taller Scooter 49cc Street Legal Free Matching...,749.00,0.000047,10,41.307692
2,226880,KUFA Sports Fighting Belt KUFA Sports Fighting...,18.53,0.000076,10,58.095238
3,165114,Stainless Steel 304 Spring Snap Hook Carabiner...,58.30,0.000036,10,88.500000
4,256154,Bushnell Trophy XLT Multi-X Reticle Riflescope...,119.95,0.000036,10,77.500000
...,...,...,...,...,...,...
45497,182494,Blue Force Gear Vickers Black 2-Pt Combat Slin...,42.12,0.000036,10,48.700000
45498,233968,Sako Optilok Scope Mount Base Sako Optilok Sco...,58.30,0.000022,2,5.166667
45499,175781,MLB New York Yankees MLB Basic Stm/Gry 59Fifty...,58.30,0.000014,4,81.000000
45500,194617,USA GunClub Easy Use Gun Hanger Pack of 4 Orig...,13.99,0.000311,11,16.441860


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from tqdm.auto import tqdm

# Assuming base_dir is defined elsewhere in your notebook
base_dir = '/content/drive/MyDrive/Data Science AUEB/Data Challenge/data/'

# Load the data
train_df = pd.read_csv(f'{base_dir}final_df.csv')
test_df = pd.read_csv(f'{base_dir}test_df.csv')

# Drop rows with missing text
train_df = train_df.dropna(subset=['text'])

# Feature selection
X_train = train_df[['text', 'price', 'node_degree', 'core_number', 'avg_neighbor_degree']]
y_train = train_df['class_label']

X_test = test_df[['text', 'price', 'node_degree', 'core_number', 'avg_neighbor_degree']]

# Text preprocessing using TF-IDF (reduced features, unigrams and bigrams)
vectorizer = TfidfVectorizer(max_features=4000, stop_words='english', ngram_range=(1, 2))

# Fit the vectorizer on train text and transform both train and test text data
X_train_text = vectorizer.fit_transform(X_train['text'])
X_test_text = vectorizer.transform(X_test['text'])

# Combine text features with the other numerical features using hstack for sparse matrices
X_train_numerical = X_train.drop(columns=['text']).values
X_test_numerical = X_test.drop(columns=['text']).values

X_train_combined = hstack([X_train_text, X_train_numerical])
X_test_combined = hstack([X_test_text, X_test_numerical])

# Train-test split (using 20% for validation)
X_train_sparse, X_val_sparse, y_train, y_val = train_test_split(X_train_combined, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Convert training and validation data to LightGBM dataset format
train_data = lgb.Dataset(X_train_sparse, label=y_train)
val_data = lgb.Dataset(X_val_sparse, label=y_val, reference=train_data) # Specify reference dataset for early stopping


In [ ]:
# Specify parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': len(y_train.unique()),  # Number of classes in the target variable
    'metric': 'multi_logloss',
    'learning_rate': 0.05,
    'num_leaves': 50,
    'max_depth': 10,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_child_samples': 20,
}

# Train the LightGBM model with early stopping
model = lgb.train(params,
                  train_data,
                  num_boost_round=1000,  # Set a large number of boosting rounds
                  valid_sets=[train_data, val_data],
                  callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=1), lgb.log_evaluation(period=100)], # Use the callbacks for early stopping and verbose evaluation
                  )

# Make predictions on the validation set
y_val_pred = model.predict(X_val_sparse, num_iteration=model.best_iteration)
log_loss_value = log_loss(y_val, y_val_pred)
print(f'Log Loss on Validation Set (with early stopping): {log_loss_value}')

# Final predictions for the test set using the trained model
test_preds = model.predict(X_test_combined, num_iteration=model.best_iteration)

# Create the submission DataFrame
submission_df = pd.DataFrame(test_preds, columns=[f'class{i}' for i in range(len(y_train.unique()))])

# Add the 'product' column (product_id from test_df)
submission_df['product'] = test_df['product_id']

# Reorder columns to ensure 'product' is the first column
submission_df = submission_df[['product'] + [f'class{i}' for i in range(len(y_train.unique()))]]

# Save the DataFrame as a CSV file for submission
submission_file_path = '/content/drive/MyDrive/Data Science AUEB/Data Challenge/data/submission_lGBM_early_stopping.csv'
submission_df.to_csv(submission_file_path, index=False)

# Show the submission DataFrame (optional)
submission_df.head()

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


,product,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12,class13,class14,class15
0,49957,1.314277e-04,0.001120,0.112206,4.928990e-05,0.002815,0.000606,0.000038,0.000023,0.000243,0.000010,0.880126,0.000727,0.001730,1.082962e-05,5.693778e-05,1.067860e-04
1,135386,7.176006e-09,0.000020,0.000059,1.128003e-07,0.000036,0.000288,0.000005,0.000005,0.000008,0.999571,0.000006,0.000001,0.000001,3.640124e-08,4.415885e-07,6.774386e-08
2,226880,2.635342e-04,0.007317,0.772666,1.110515e-04,0.074407,0.002228,0.012366,0.000805,0.000118,0.000054,0.122471,0.000556,0.006584,2.527399e-06,6.633745e-06,4.475121e-05
3,165114,2.502210e-05,0.004143,0.092623,3.256793e-05,0.026645,0.002514,0.000625,0.000510,0.000072,0.000081,0.871000,0.000156,0.001528,2.693464e-05,1.407620e-05,3.391528e-06
4,256154,8.762545e-08,0.000108,0.998845,9.833096e-07,0.000129,0.000251,0.000008,0.000104,0.000003,0.000002,0.000419,0.000016,0.000114,2.161231e-07,2.120354e-07,2.709178e-07
